In [2]:
import os
import pickle
import numpy as np
from scipy import signal
from scipy.signal import butter, iirnotch, lfilter
import numpy as np
import matplotlib.pyplot as plt

from os import listdir
from os.path import isfile, join, isdir
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
import scipy.ndimage

#Extracting the data from the 
data_set_path = "D:\Downloads\WESAD\WESAD\\"
subject = ["S2",'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S13', 'S14', 'S15', 'S16', 'S17']

class read_data_of_one_subject:
    """Read data from WESAD dataset"""
    def __init__(self, path, subject):
        self.keys = ['label', 'subject', 'signal']
        self.signal_keys = ['wrist', 'chest']
        self.chest_sensor_keys = ['ACC', 'ECG', 'EDA', 'EMG', 'Resp', 'Temp']
        self.wrist_sensor_keys = ['ACC', 'BVP', 'EDA', 'TEMP']
        #os.chdir(path)
        #os.chdir(subject)
        with open(path + subject +'/'+subject + '.pkl', 'rb') as file:
            data = pickle.load(file, encoding='latin1')
        self.data = data

    def get_labels(self):
        return self.data[self.keys[0]]

    def get_wrist_data(self):
        """"""
        #label = self.data[self.keys[0]]
        assert subject == self.data[self.keys[1]]
        signal = self.data[self.keys[2]]
        wrist_data = signal[self.signal_keys[0]]
        #wrist_ACC = wrist_data[self.wrist_sensor_keys[0]]
        #wrist_ECG = wrist_data[self.wrist_sensor_keys[1]]
        return wrist_data

    def get_chest_data(self):
        """"""
        signal = self.data[self.keys[2]]
        chest_data = signal[self.signal_keys[1]]
        return chest_data

In [4]:
from ECG import ECGprep
import sys
sys.path.insert(1, 'D:\Documents\GitHub\EPO-4_BioBombs\Machine learning')

eda_features = np.asarray([0,0,0,0], dtype = "float")
 

#Train set
for i in range(len(subject)):
    obj_data = {}

    obj_data[subject[i]] = read_data_of_one_subject(data_set_path, subject[i])
    #print(obj_data[subject[i]].data)
    chest_data_dict = obj_data[subject[i]].get_chest_data()

    labels = obj_data[subject[i]].get_labels() 
    baseline = np.asarray([idx for idx,val in enumerate(labels) if val == 1])
    stress = np.asarray([idx for idx,val in enumerate(labels) if val == 2])

    ecg_stress=chest_data_dict['ECG'][stress,0]
    ecg_base=chest_data_dict['ECG'][baseline,0]

    fs=700
    # cut a smaller window
    t_tot=(len(stress)//(int(0.5*60*fs)))
    ecg_stress_tot=np.empty([21000,t_tot])
    ecg_base_tot=np.empty([21000,t_tot])

    for i in range(t_tot): 
        ecg1=ecg_stress[i*int(0.5*60*700):(i+1)*int(0.5*60*700)]
        ecg2=ecg_base[i*int(0.5*60*700):(i+1)*int(0.5*60*700)]
        t1=np.arange(0,ecg1.size*(1/fs),(1/fs))
        t1=t1[:ecg1.size]
        t2=np.arange(0,ecg2.size*(1/fs),(1/fs))
        t2=t2[:ecg2.size]
        ecg_stress_tot[:,i] = ecg1
        ecg_base_tot[:,i]=ecg2
        
    #print(eda1.shape)
    #print(eda_stress_tot.shape)

    #print(eda2.shape)
    #print(eda_stress_tot.shape)

    ecg_comp_base=np.zeros((3,11000,t_tot))
    ecg_comp_stress=np.zeros((3,11000,t_tot))
    ECG_base = []
    ECG_stress = []

    for i in range (t_tot): 
        ECG_base = ECGprep(fs, ecg_base_tot[:,i],"baseline")
        ECG_stress = ECGprep(fs, ecg_stress_tot[:,i],"stress")

        ecg_plot=ECG_base.plotdata()
        ecg_plot2=ECG_stress.plotdata()
        ecg_filt = ECG_base.filtering_data()

        '''eda_lp = EDA_base.filtering_data()
        eda_sm = EDA_base.smoothing_data(eda_lp)
        eda_comp_base[:,:,i]=EDA_base.decompose_data(eda_sm)

        eda_lp = EDA_stress.filtering_data()
        eda_sm = EDA_stress.smoothing_data(eda_lp)
        eda_comp_stress[:,:,i]=EDA_stress.decompose_data(eda_sm)

        t=np.arange(0,phasic_stress.size*(1/fs),(1/fs))
        t=t[:phasic_stress.shape[0]]'''

        # plt.figure(figsize=(12,4))
        # plt.xlim([0,30])
        # plt.plot(t,phasic_stress,label='phasic')
        # plt.xlabel('$Time (s)$') 
        # plt.ylabel('$EDA$') 
        # plt.legend()

        #print((phasic_stress[0:10]))

        #For the state, True is stress and False is base
        '''
        eda_features = np.vstack((eda_features,stress_feature))
        eda_features = np.vstack((eda_features,calc_phasic_features(phasic_base, t_tot, False)))'''


print(np.shape(eda_features))
print(eda_features)



TypeError: ECGprep.__init__() takes from 2 to 4 positional arguments but 5 were given